In [ ]:
using Turing
using StatsPlots
using Optim

In [ ]:
PRINT_PROGRESS = true
include("read_data.jl")
include("utils.jl")
include("lecture/plot_utils.jl");

### Dataset: World-Wide Monthly Air Passengers

In [ ]:
base_plot(air_passengers_2013_2018)

In [ ]:
# upwards_trend_data = air_passengers_2013_2018[2009 .<= air_passengers_2013_2018[!,"Year"],:]
# base_plot(upwards_trend_data)

# x = upwards_trend_data[!,"Date"]
# y = upwards_trend_data[!,"Total"];

x = air_passengers_2013_2018[!,"Date"]
y = air_passengers_2013_2018[!,"Total"];

In [ ]:
@model function trend_model(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(2,3)
    tau ~ InverseGamma(1,1)

    # UPGRADE: fix one changepoint per year, put restrictive Laplace prior on adjustments
    n_changepoints = length(y) ÷ 12
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    
    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        # UPGRADE: adjust slope and intercept at beginning of each year
        if i % 12 == 1
            k += adjustments[j]
            m -= (x[i] - x[1]) * adjustments[j]
            j += 1
        end
        y[i] ~ Normal(k * (x[i] - x[1]) + m, error+1e-5)
    end
end

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model(x, y), NUTS(), 3000, progress=PRINT_PROGRESS)

In [ ]:
Turing.Random.seed!(0)
res = sample(trend_model(x,y), NUTS(), 3000, progress=PRINT_PROGRESS);

In [ ]:
_, map_vector = get_map(res)
plot_trend_model_3(air_passengers_2013_2018, map_vector)

In [ ]:
res = optimize(trend_model(x, y), MAP(), Optim.Options(iterations=10_000))

In [ ]:
map_vector = res.values.array
plot_trend_model_3(air_passengers_2013_2018, map_vector)